# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Get the PRICE_DATA variable
price_data_path = os.getenv("PRICE_DATA")

# Print to verify
print(f"PRICE_DATA path: {price_data_path}")

PRICE_DATA path: ../../05_src/data/prices/


In [5]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
from glob import glob

# Load environment variables
load_dotenv()

# Get the PRICE_DATA directory path
price_data_path = os.getenv("PRICE_DATA")

# Find all parquet files in the directory
parquet_files = glob(os.path.join(price_data_path, "*.parquet"))

# Print to verify
print(parquet_files)


[]


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [43]:
import dask.dataframe as dd

# Define the file location
file_location = "/Users/atulkalra/Desktop/DSI Assignment/production/05_src/data/temp/stock_px.parquet"

# Load the data into a Dask DataFrame from the Parquet file
dd_data = dd.read_parquet(file_location)

# Print out the column names to verify what's available
print("Columns in dd_data:", dd_data.columns)

# Check if the 'date' column exists. If not, check if 'Date' exists and rename it.
if 'date' not in dd_data.columns:
    if 'Date' in dd_data.columns:
        print("Renaming 'Date' to 'date'")
        dd_data = dd_data.rename(columns={'Date': 'date'})
    else:
        print("No date column found. 'date' will be omitted from the final selection.")

# Sort by date (and ticker if available)
if 'ticker' in dd_data.columns and 'date' in dd_data.columns:
    dd_data = dd_data.sort_values(by=['ticker', 'date'])
elif 'date' in dd_data.columns:
    dd_data = dd_data.sort_values(by=['date'])
else:
    # If no date column is available, sort by ticker if possible, or skip sorting.
    if 'ticker' in dd_data.columns:
        dd_data = dd_data.sort_values(by=['ticker'])

# Add lag for 'Close'
dd_data['Close_lag_1'] = dd_data['Close'].shift(1)

# Add lag for 'Adj Close'
dd_data['Adj_Close_lag_1'] = dd_data['Adj Close'].shift(1)

# Add returns based on 'Close'
dd_data['returns'] = (dd_data['Close'] / dd_data['Close_lag_1']) - 1

# Add high-low range for each row
dd_data['hi_lo_range'] = dd_data['High'] - dd_data['Low']

# Build the list of columns for the final DataFrame
selected_columns = []

if 'ticker' in dd_data.columns:
    selected_columns.append('ticker')

if 'date' in dd_data.columns:
    selected_columns.append('date')

selected_columns += ['Close', 'Adj Close', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range']

# Assign the result to dd_feat
dd_feat = dd_data[selected_columns]

# Print the first few rows to verify the results
print(dd_feat.head())

Columns in dd_data: Index(['Date', 'Ticker', 'Adj Close', 'Close', 'High', 'Low', 'Open',
       'Volume'],
      dtype='object')
Renaming 'Date' to 'date'
                       date      Close  Adj Close  Close_lag_1  \
0 2000-01-03 00:00:00+00:00   0.999442   0.843076          NaN   
1 2000-01-03 00:00:00+00:00        NaN        NaN     0.999442   
2 2000-01-03 00:00:00+00:00  58.281250  35.864075          NaN   
3 2000-01-03 00:00:00+00:00   0.097526   0.089440    58.281250   
4 2000-01-04 00:00:00+00:00   0.915179   0.771997     0.097526   

   Adj_Close_lag_1   returns  hi_lo_range  
0              NaN       NaN     0.096540  
1         0.843076       NaN          NaN  
2              NaN       NaN     3.312500  
3        35.864075 -0.998327     0.007292  
4         0.089440  8.383949     0.084263  


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [44]:
import pandas as pd

# Convert Dask DataFrame to Pandas DataFrame
dd_feat_pd = dd_feat.compute()

# Add a new feature for the moving average of returns using a 10-day window
dd_feat_pd['returns_10_day_ma'] = dd_feat_pd['returns'].rolling(window=10).mean()

# Print the first few rows to verify the new feature
print(dd_feat_pd.head())

                       date      Close  Adj Close  Close_lag_1  \
0 2000-01-03 00:00:00+00:00   0.999442   0.843076          NaN   
1 2000-01-03 00:00:00+00:00        NaN        NaN     0.999442   
2 2000-01-03 00:00:00+00:00  58.281250  35.864075          NaN   
3 2000-01-03 00:00:00+00:00   0.097526   0.089440    58.281250   
4 2000-01-04 00:00:00+00:00   0.915179   0.771997     0.097526   

   Adj_Close_lag_1   returns  hi_lo_range  returns_10_day_ma  
0              NaN       NaN     0.096540                NaN  
1         0.843076       NaN          NaN                NaN  
2              NaN       NaN     3.312500                NaN  
3        35.864075 -0.998327     0.007292                NaN  
4         0.089440  8.383949     0.084263                NaN  


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It wasn't necessary to convert to Pandas to calculate the moving average return, as Dask supports rolling window operations. However, performing the calculation in Dask would have been more efficient for larger datasets, as it allows parallel processing and avoids memory limitations. Converting to Pandas may be suitable for smaller datasets but could lead to memory issues with larger ones. Dask is better for scalability and handling larger-than-memory data.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.